Chapter 10: Data Aggregation and Group Operations

In [2]:
import numpy as np
import pandas as pd

Group operations follow a split-apply-combine sequence. First data is split into groups based on one or more keys (either split across rows or columns), then a function is applied to each group, then the results are combined into a result.

In [3]:
# Start with a tabular dataset made into a DataFrame
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-0.025018,2.228835
1,a,2,-0.359978,-0.031013
2,None,1,-1.526958,0.751707
3,b,2,-1.519590,-1.216182
4,b,1,0.438531,-1.875699
5,a,<NA>,1.576002,-0.229891
6,None,1,0.985402,0.146828


In [4]:
# Sort the df by picking out the data1 column and group by key1
grouped = df["data1"]
grouped

0   -0.025018
1   -0.359978
2   -1.526958
3   -1.519590
4    0.438531
5    1.576002
6    0.985402
Name: data1, dtype: float64

In [5]:
grouped = df["data1"].groupby(df["key1"])
grouped

In [6]:
# The groupby object has all the info needed to apply an operation to each of the groups
grouped.mean()

key1
a    0.397002
b   -0.540529
Name: data1, dtype: float64

In [7]:
# Passing multiple arrays to "group by" gives different groups and different values
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1      -0.025018
      2      -0.359978
b     1       0.438531
      2      -1.519590
Name: data1, dtype: float64

In [8]:
# You can group by any arrays of the correct length
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]

df["data1"].groupby([states, years]).mean()

CA  2005    0.608012
    2006   -1.526958
OH  2005   -0.772304
    2006    0.711967
Name: data1, dtype: float64

In [9]:
# size returns a Series containing group sizes
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [10]:
df.groupby("key1").size()

key1
a    3
b    2
dtype: int64

In [11]:
df.groupby("key1", dropna=False).size()

key1
a      3
b      2
NaN    2
dtype: int64

In [12]:
# The object returned by groupby supports iteration
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1 -0.025018  2.228835
1    a     2 -0.359978 -0.031013
5    a  <NA>  1.576002 -0.229891
b
  key1  key2     data1     data2
3    b     2 -1.519590 -1.216182
4    b     1  0.438531 -1.875699


In [13]:
# This allows for the computation of a dictionary of the data pieces as a one-liner:
pieces = {name: group for name, group in df.groupby("key1")}
pieces

{'a':   key1  key2     data1     data2
 0    a     1 -0.025018  2.228835
 1    a     2 -0.359978 -0.031013
 5    a  <NA>  1.576002 -0.229891,
 'b':   key1  key2     data1     data2
 3    b     2 -1.519590 -1.216182
 4    b     1  0.438531 -1.875699}

In [14]:
# We can group the columns of the dataframe by whether they start with "key" or "data"
grouped = df.groupby({"key1": "key", "key2" : "key", "data1": "data", "data2": "data"}, axis="columns")
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0 -0.025018  2.228835
1 -0.359978 -0.031013
2 -1.526958  0.751707
3 -1.519590 -1.216182
4  0.438531 -1.875699
5  1.576002 -0.229891
6  0.985402  0.146828
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


In [15]:
# Compute the mean only for the data2 column and return the result as a dataframe
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     2.228835
     2    -0.031013
b    1    -1.875699
     2    -1.216182

In [16]:
# Grouping with Dictionaries and Series
# make new df
people = pd.DataFrame(np.random.standard_normal((5, 5)), columns=["a", "b", "c", "d", "e"], index=["Joe", "Steve", "Wanda", "Jill", "Trey"])
people

,a,b,c,d,e
Joe,0.527618,0.701716,-1.617540,0.110221,-1.779590
Steve,-0.312357,-1.028041,-0.300719,2.131275,-0.739694
Wanda,-0.772981,-0.577427,0.266052,-1.012105,-1.428784
Jill,0.066752,0.634038,0.098585,-0.243004,0.467702
Trey,-0.947735,0.333742,-0.819854,-0.374790,0.900365


In [17]:
# add NA values
people.iloc[2:4, [1,2]] = np.nan
people

,a,b,c,d,e
Joe,0.527618,0.701716,-1.617540,0.110221,-1.779590
Steve,-0.312357,-1.028041,-0.300719,2.131275,-0.739694
Wanda,-0.772981,NaN,NaN,-1.012105,-1.428784
Jill,0.066752,NaN,NaN,-0.243004,0.467702
Trey,-0.947735,0.333742,-0.819854,-0.374790,0.900365


In [18]:
# Create a dictionary containing a mapping
mapping = {"a": "red", "b": "red", "c": "blue", "d": "blue", "e": "red", "f": "orange"}
mapping
# The mapping "f" above will get ignored

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [19]:
# Pass the dictionary to "groupby"
by_column = people.groupby(mapping, axis="columns")
by_column.sum()

,blue,red
Joe,-1.507320,-0.550257
Steve,1.830556,-2.080092
Wanda,-1.012105,-2.201765
Jill,-0.243004,0.534454
Trey,-1.194644,0.286372


In [20]:
# We can also pass the dictionary to a series
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [21]:
people.groupby(map_series, axis="columns").count()

,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,1,2
Trey,2,3


In [22]:
# Grouping with Functions

# Group the previous dataframe by passing the len function to groupby
people.groupby(len).sum()

,a,b,c,d,e
3,0.527618,0.701716,-1.617540,0.110221,-1.779590
4,-0.880983,0.333742,-0.819854,-0.617794,1.368067
5,-1.085338,-1.028041,-0.300719,1.119171,-2.168477


In [23]:
# You can also pass a mixed function/array/dict/Series to groupby
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,0.527618,0.701716,-1.617540,0.110221,-1.779590
4,two,-0.947735,0.333742,-0.819854,-0.374790,0.467702
5,one,-0.772981,-1.028041,-0.300719,-1.012105,-1.428784


In [24]:
# Data Aggregation
# An aggregation is a data transformation which produces a scalar from an array (i.e. mean, count, min, sum)
df

,key1,key2,data1,data2
0,a,1,-0.025018,2.228835
1,a,2,-0.359978,-0.031013
2,None,1,-1.526958,0.751707
3,b,2,-1.519590,-1.216182
4,b,1,0.438531,-1.875699
5,a,<NA>,1.576002,-0.229891
6,None,1,0.985402,0.146828


In [25]:
# Group by key1
grouped = df.groupby("key1")
# Find 2 smallest values in the grouped df under "data1" column
grouped["data1"].nsmallest(2)

key1   
a     1   -0.359978
      0   -0.025018
b     3   -1.519590
      4    0.438531
Name: data1, dtype: float64

In [26]:
# You can define your own functions which act on the whole array and pass them to agg
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,1.935979,2.458726
b,1,1.958121,0.659517


In [27]:
# Return to tipping dataset
tips = pd.read_csv("datasets/tips.csv")
print("Shape of data frame: {}".format(tips.shape))
tips.head()

Shape of data frame: (244, 6)


,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [28]:
# Add a tip percentage column
tips["tip_pct"] = tips["tip"]/tips["total_bill"]
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [29]:
# Group the tips by day and smoking status
grouped = tips.groupby(["day", "smoker"])
# Pull the tip percentage column
grouped_pct = grouped["tip_pct"]
# Pass the name of the function as a string to agg
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [30]:
# Can also pass a list of functions
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [31]:
# Can give the returns from the functions new names
grouped_pct.agg([("average", "mean"), ("stdev", np.std)])

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [32]:
# For data frames, can pass the same function to different columns or different functions to different columns
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [33]:
# To apply different functions to different columns, pass a dictionary to agg
grouped.agg({"tip": np.max, "size": "sum"})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [34]:
grouped.agg({"tip_pct" : ["min", "max", "mean", "std"], "size" : "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [35]:
# Can change the index from grouped keys to a flattened index
grouped = tips.groupby(["day", "smoker"], as_index=False)
grouped.mean(numeric_only=True)

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [36]:
# The apply method splits the object it acts on into pieces, invokes a function on each piece, and then concatenates the pieces together.
# Here we want to select the top five tip percentage values by group

# Define a function
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [37]:
# Now group by "smoker" and call apply with the top function
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [38]:
# Functions with optional inputs can also be passed
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [39]:
# Again, you can flatten the indexes
tips.groupby("smoker", group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [40]:
# pandas makes it easy to do quantile analysis on a dataset
# Make new dataset
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,0.399374,-0.460397
1,-0.465477,-1.541707
2,-0.998959,1.016067
3,-0.652824,-1.214728
4,0.847355,-0.259875


In [41]:
# Now find the quartiles for the "data1" column
quartiles = pd.cut(frame["data1"], 4)
quartiles.head(10)

0       (-0.311, 1.5]
1    (-2.122, -0.311]
2    (-2.122, -0.311]
3    (-2.122, -0.311]
4       (-0.311, 1.5]
5       (-0.311, 1.5]
6       (-0.311, 1.5]
7       (-0.311, 1.5]
8    (-2.122, -0.311]
9    (-2.122, -0.311]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.94, -2.122] < (-2.122, -0.311] < (-0.311, 1.5] < (1.5, 3.311]]

In [42]:
# We can then pass the quartiles object to the groupby function
def get_stats(group):
    return pd.DataFrame({"min": group.min(), "max": group.max(), "count": group.count(), "mean": group.mean()})

grouped = frame.groupby(quartiles)
grouped.apply(get_stats)

min       max  count      mean
data1                                                      
(-3.94, -2.122]  data1 -3.932616 -2.163693     21 -2.478327
                 data2 -3.321138  1.912407     21 -0.542585
(-2.122, -0.311] data1 -2.056218 -0.310675    353 -0.917919
                 data2 -3.061189  2.750965    353 -0.007739
(-0.311, 1.5]    data1 -0.308924  1.498250    565  0.484890
                 data2 -3.174061  2.794313    565  0.054024
(1.5, 3.311]     data1  1.515226  3.311329     61  2.039633
                 data2 -2.364848  1.632772     61 -0.102050

In [43]:
# But note we can do that more easily
grouped.agg(["min", "max", "count", "mean"])

data1                               data2            \
                       min       max count      mean       min       max   
data1                                                                      
(-3.94, -2.122]  -3.932616 -2.163693    21 -2.478327 -3.321138  1.912407   
(-2.122, -0.311] -2.056218 -0.310675   353 -0.917919 -3.061189  2.750965   
(-0.311, 1.5]    -0.308924  1.498250   565  0.484890 -3.174061  2.794313   
(1.5, 3.311]      1.515226  3.311329    61  2.039633 -2.364848  1.632772   

                                  
                 count      mean  
data1                             
(-3.94, -2.122]     21 -0.542585  
(-2.122, -0.311]   353 -0.007739  
(-0.311, 1.5]      565  0.054024  
(1.5, 3.311]        61 -0.102050

In [44]:
# cut returns equal length buckets, qcut returns equal sized buckets
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)
quartiles_samp.head()

0    2
1    1
2    0
3    0
4    3
Name: data1, dtype: int64

In [45]:
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count      mean
data1                                           
0     data1 -3.932616 -0.652022    250 -1.264716
      data2 -3.321138  2.750965    250 -0.044871
1     data1 -0.651729  0.052214    250 -0.317883
      data2 -3.061189  2.625057    250  0.078520
2     data1  0.052909  0.703420    250  0.358233
      data2 -2.874523  2.764851    250  0.008900
3     data1  0.707418  3.311329    250  1.313606
      data2 -3.174061  2.794313    250 -0.001858

In [46]:
# Fill NA values in a Series or DataFrame
s = pd.Series(np.random.standard_normal(6))
s[::2] = np.nan
s

0         NaN
1   -0.901689
2         NaN
3    1.993469
4         NaN
5   -1.222117
dtype: float64

In [47]:
# Fill NA values with the mean of other values
s.fillna(s.mean())

0   -0.043446
1   -0.901689
2   -0.043446
3    1.993469
4   -0.043446
5   -1.222117
dtype: float64

In [48]:
# We can also fillna on specific group elements
states= ["Ohio", "New York", "Vermont", "Florida", "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East", "West", "West", "West", "West"]
data = pd.Series(np.random.standard_normal(8), index=states)
data

Ohio          0.760519
New York     -0.031002
Vermont      -0.568803
Florida       0.312116
Oregon        1.433028
Nevada       -0.254281
California    0.999407
Idaho         0.396823
dtype: float64

In [49]:
# Insert NAN values
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio          0.760519
New York     -0.031002
Vermont            NaN
Florida       0.312116
Oregon        1.433028
Nevada             NaN
California    0.999407
Idaho              NaN
dtype: float64

In [50]:
# Look at some groupby sizes
data.groupby(group_key).size()

East    4
West    4
dtype: int64

In [51]:
data.groupby(group_key).count()

East    3
West    2
dtype: int64

In [52]:
data.groupby(group_key).mean()

East    0.347211
West    1.216217
dtype: float64

In [53]:
# Fill NA values with group means
def fill_mean(group):
    return group.fillna(group.mean())

data.groupby(group_key).apply(fill_mean)

Ohio          0.760519
New York     -0.031002
Vermont       0.347211
Florida       0.312116
Oregon        1.433028
Nevada        1.216217
California    0.999407
Idaho         1.216217
dtype: float64

In [54]:
# We can also use predefined fill values
fill_values = {"East": 0.5, "West": -1}

def fill_func(group):
    return group.fillna(fill_values[group.name])

data.groupby(group_key).apply(fill_func)

Ohio          0.760519
New York     -0.031002
Vermont       0.500000
Florida       0.312116
Oregon        1.433028
Nevada       -1.000000
California    0.999407
Idaho        -1.000000
dtype: float64

In [55]:
# Example of random sampling and permutation
# Make a deck of cards
suits = ["H", "S", "C", "D"]
card_val = (list(range(1,11)) + [10]*3)*4
base_names = ["A"] + list(range(2,11)) + ["J", "Q", "K"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index = cards)
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
QH     10
KH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
QS     10
KS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
QC     10
KC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
QD     10
KD     10
dtype: int64

In [56]:
# Now simulate drawing five cards
def draw(deck, n=5):
    return deck.sample(n)

draw(deck)

10S    10
9H      9
5H      5
9S      9
5D      5
dtype: int64

In [57]:
# Draw two cards of the same suit

def get_suit(card):
    return card[-1]

deck.groupby(get_suit).apply(draw,n=2)

C  2C    2
   9C    9
D  5D    5
   8D    8
H  4H    4
   8H    8
S  8S    8
   AS    1
dtype: int64

In [58]:
# Compute group weighted averages and correlations
df = pd.DataFrame({"category": ["a", "a", "a", "a",
                                "b", "b", "b", "b"],
                   "data": np.random.standard_normal(8),
                   "weights": np.random.uniform(size=8)})
df

,category,data,weights
0,a,0.848323,0.958737
1,a,0.448661,0.602543
2,a,-0.034157,0.874121
3,a,-0.748050,0.373672
4,b,0.264207,0.885992
5,b,1.535900,0.388615
6,b,1.712059,0.016559
7,b,-0.012603,0.125150


In [59]:
# Compute the weighted average by category
grouped = df.groupby("category")

def get_wavg(group):
    return np.average(group["data"], weights = group["weights"])

grouped.apply(get_wavg)

category
a    0.275633
b    0.605608
dtype: float64

Group Transforms and "Unwrapped" GroupBys

In [60]:
# Practice with build-in transform method
df = pd.DataFrame({'key': ['a', 'b', 'c']*4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [62]:
# Group means by key
g = df.groupby('key')['value']
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [63]:
# Now, say we want a Series of the same shape as df['value'] but with values replaced by the average grouped by 'key'
def get_mean(group):
    return group.mean()

g.transform(get_mean)

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [64]:
# We can also used built-in aggregation functions
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [65]:
# More examples
def times_two(group):
    return group*2

g.transform(times_two)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [66]:
def get_ranks(group):
    return group.rank(ascending=False)

g.transform(get_ranks)

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [67]:
def normalize(x):
    return (x-x.mean())/x.std()

g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

Pivot Tables and Cross-Tabulation

- Pivot tables aggregate a table of data by one or more keys

In [68]:
# Return to tipping dataset, compute a table of group means arranged by day and smoker on the rows
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [69]:
tips.pivot_table(index=["day", "smoker"],
                 values=["size", "tip", "tip_pct", "total_bill"])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [70]:
# Take the average of only tip_pct and size and group by time
tips.pivot_table(index=["time", "day"], columns="smoker", values=["tip_pct", "size"])

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [73]:
# Cross tabulations (or crosstabs) are a special case of a pivot table that computes group frequencies
from io import StringIO

data = """Sample  Nationality  Handedness
 1   USA  Right-handed
 2   Japan    Left-handed
 3   USA  Right-handed
 4   Japan    Right-handed
 5   Japan    Left-handed
 6   Japan    Right-handed
 7   USA  Right-handed
 8   USA  Left-handed
 9   Japan    Right-handed
 10  USA  Right-handed"""

data = pd.read_table(StringIO(data), sep="\s+")
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [74]:
# Summarize by nationality and handedness
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10
